In [ ]:
import os
from PIL import Image
import numpy as np
import albumentations as A
import xml.etree.ElementTree as ET
from matplotlib import pyplot as plt
import cv2
import shutil
%matplotlib inline

In [ ]:
folder_path = os.path.join('D:\\','OneDrive', 'Tensorflow', 'Image_0306','train3')

In [ ]:
dirs = os.listdir( folder_path )

In [ ]:
class_labels =  ['Wood']

In [ ]:
def get_boxes_coordinate(file) :
    coordinate_boxes = []
    tree = ET.parse(file)
    root = tree.getroot()
    for child in root:
        if child.tag == "object":
            box = child[4]
            if box is not None:
                ordinate = [box[0].text,box[0].text,box[0].text,box[0].text,class_labels[0]]
                coordinate_boxes.append(ordinate)
    return coordinate_boxes
            

In [ ]:
def override_xml_content(file,img_file_name,image_file_path) :
    tree = ET.parse(file)
    root = tree.getroot()
    root[1].text = img_file_name
    root[2].text = image_file_path
    tree.write(file)

In [ ]:
random_weather = {
    "RandomRain" : lambda : A.RandomRain(brightness_coefficient = 0.9,drop_width=1,blur_value =5 , p = 1 ),
#     "RandomSnow" :  lambda : A.RandomSnow(brightness_coeff=2.5,snow_point_lower=0.3,snow_point_upper=0.5,p=1),
    "RandomSunFlare" :  lambda : A.RandomSunFlare(flare_roi=(0, 0, 1, 0.5),angle_lower = 0.5,p=1),
    "RandomShadow" :  lambda : A.RandomShadow(num_shadows_lower=1,num_shadows_upper=1,shadow_dimension=5,shadow_roi=(0, 0.5, 1, 1),p=1),
#     "RandomFog" :  lambda : A.RandomFog(fog_coef_lower=0.7,fog_coef_upper=0.8,p=1),
#     "Light" : [lambda :A.RandomBrightnessContrast(p=1),lambda :A.RandomGamma(p=1),lambda :A.CLAHE(p=1)],
#     "HueSaturation": lambda : A.ChannelShuffle(p=1),
}

In [ ]:
for item in dirs:
        file_path = folder_path+"\\"+item
        if os.path.isfile(file_path):
            if file_path.endswith(('.png', '.jpg', '.jpeg')) :
                image = cv2.imread(file_path)
                bboxes = get_boxes_coordinate(file_path.replace('jpg','xml'))
                for attr, value in random_weather.items() : 
                    compose_inside = []
                    if type(value) == list : 
                        for val in value :
                            compose_inside.append(val())
                    else :
                        compose_inside.append(value())
                    transform = A.Compose(compose_inside, bbox_params=A.BboxParams(format='pascal_voc'))
                    transformed = transform(image=image, bboxes=bboxes)
                    new_img_aumentation_name = item.replace('.jpg','')+ "_" + attr + '.jpg'
                    augmentation_name = folder_path + "\\" + new_img_aumentation_name
                    print("save file to : {0}".format(augmentation_name))
                    xml_item = item.replace('jpg','xml')
                    print(xml_item)
                    new_xml_name = xml_item.replace('.xml','')+ "_"+attr + '.xml'
                    new_xml_path = folder_path+"\\" + new_xml_name
                    shutil.copyfile(folder_path+"\\"+xml_item, new_xml_path)
                    override_xml_content(new_xml_path,new_img_aumentation_name,augmentation_name)
                    cv2.imwrite(augmentation_name,transformed['image'])